# Surprise 라이브러리 활용한 추천 시스템 구축
- 버전 체크
- NumPy 버전을 1.26.4 설정 (최신 버전 2.3), 환경설정 할 때 주의

In [1]:
import surprise 
print(surprise.__version__)

1.1.4


# 라이브러리 불러오기

In [7]:
from surprise import SVD
from surprise import Dataset
from surprise import accuracy
from surprise.model_selection import train_test_split 

# 데이터 불러오기
data = Dataset.load_builtin('ml-100k')

# 가공, surprise.dataset.DatasetAutoFolds ==> pandas DataFrame으로 변경 (옵션)
# 데이터 엔지니어, 분석가들이 해야 하는 역할


# 데이터 분할 
trainset, testset = train_test_split(data, test_size=.25, random_state=0)

# 모델링

In [8]:
algo = SVD(random_state=0)
algo.fit(trainset)

# 테스트

In [14]:
predictions = algo.test(testset)
print(len(predictions), type(predictions))
predictions[:3]

25000 <class 'list'>


[Prediction(uid='120', iid='282', r_ui=4.0, est=3.5114147666251547, details={'was_impossible': False}),
 Prediction(uid='882', iid='291', r_ui=4.0, est=3.573872419581491, details={'was_impossible': False}),
 Prediction(uid='535', iid='507', r_ui=5.0, est=4.033583485472447, details={'was_impossible': False})]

In [15]:
# 프로젝트 할 때, 결괏값을 pandas 데이터프레임으로 만들고 나서 DB에 저장, CSV로 내보내기 하거나 등 작업을 수행하면 됨
# 각 개별적인 인수에 접근하는 방식 기억
for pred in predictions[:3]:
    print(pred.uid, pred.iid, pred.est, pred.details)

120 282 3.5114147666251547 {'was_impossible': False}
882 291 3.573872419581491 {'was_impossible': False}
535 507 4.033583485472447 {'was_impossible': False}


In [16]:
# 예시 
uid = str(120)  # 숫자를 문자열로 형변환
iid = str(282)  # 숫자를 문자열로 형변환
pred = algo.predict(uid, iid)
print(pred)

user: 120        item: 282        r_ui = None   est = 3.51   {'was_impossible': False}


# 평가

In [17]:
accuracy.rmse(predictions)

RMSE: 0.9467


0.9466860806937948

# 신규 데이터로 추천시스템

In [19]:
import pandas as pd 

ratings = pd.read_csv('data/ml-latest-small/ratings.csv')
ratings.to_csv('data/ml-latest-small/ratings_noh2.csv', index=False, header=False)

In [23]:
from surprise import Reader, Dataset 
reader = Reader(line_format='user item rating timestamp', sep=',', rating_scale=(0.5, 5))
data = Dataset.load_from_file('data/ml-latest-small/ratings_noh.csv', reader=reader)

trainset, testset = train_test_split(data, test_size=.25, random_state=0)

algo = SVD(n_factors=50, random_state=0)

# 학습 데이터 세트로 학습 후, 테스트 데이터 적용
algo.fit(trainset)
predictions = algo.test(testset)
accuracy.rmse(predictions)

RMSE: 0.8682


0.8681952927143516

# 데이터 활용

In [26]:
import pandas as pd 
from surprise import Reader, Dataset 

ratings = pd.read_csv('data/ml-latest-small/ratings.csv') # 컬럼 존재
reader = Reader(rating_scale=(0.5, 5))

# ratings DataFrame에서 컬럼은 사용자 아이디, 아이템 아이디, 평점 순서 
data = Dataset.load_from_df(ratings[['userId', 'movieId', 'rating']], reader)

trainset, testset = train_test_split(data, test_size=.25, random_state=0)
algo = SVD(n_factors=50, random_state=0)

# 학습 데이터 세트로 학습 후, 테스트 데이터 적용
algo.fit(trainset)
predictions = algo.test(testset)
accuracy.rmse(predictions)

RMSE: 0.8682


0.8681952927143516

# 교차 검증과 하이퍼 파라미터 튜닝

In [27]:
import pandas as pd 
from surprise import Reader, Dataset 
from surprise.model_selection import cross_validate

ratings = pd.read_csv('data/ml-latest-small/ratings.csv') # 컬럼 존재
reader = Reader(rating_scale=(0.5, 5))

# ratings DataFrame에서 컬럼은 사용자 아이디, 아이템 아이디, 평점 순서 
data = Dataset.load_from_df(ratings[['userId', 'movieId', 'rating']], reader)
algo = SVD(random_state=0)
cross_validate(algo, data, measures=['RMSE', 'MAE'], cv=5, verbose=True)

Evaluating RMSE, MAE of algorithm SVD on 5 split(s).

                  Fold 1  Fold 2  Fold 3  Fold 4  Fold 5  Mean    Std     
RMSE (testset)    0.8733  0.8717  0.8655  0.8799  0.8811  0.8743  0.0057  
MAE (testset)     0.6720  0.6705  0.6641  0.6773  0.6762  0.6720  0.0047  
Fit time          0.44    0.43    0.43    0.39    0.41    0.42    0.02    
Test time         0.04    0.04    0.04    0.04    0.04    0.04    0.00    


{'test_rmse': array([0.87332827, 0.87165013, 0.86550618, 0.87986284, 0.88114387]),
 'test_mae': array([0.67199708, 0.67052009, 0.66409951, 0.67733718, 0.67615924]),
 'fit_time': (0.4375894069671631,
  0.43001556396484375,
  0.4310905933380127,
  0.3860933780670166,
  0.4079935550689697),
 'test_time': (0.043997764587402344,
  0.044985055923461914,
  0.040946245193481445,
  0.04200029373168945,
  0.039000511169433594)}

In [28]:
from surprise.model_selection import GridSearchCV
param_grid = {'n_epochs': [20, 40, 60], 'n_factors': [50, 100, 200]}

gs = GridSearchCV(SVD, param_grid, measures=['rmse', 'mae'], cv=3)
gs.fit(data)

print(gs.best_score['rmse'])
print(gs.best_params['rmse'])

0.877334127095764
{'n_epochs': 20, 'n_factors': 50}


# DatasetAutoFolds 클래스 활용

In [31]:
from surprise.dataset import DatasetAutoFolds

reader = Reader(line_format='user item rating timestamp', sep=',', rating_scale=(0.5, 5))
data_folds = DatasetAutoFolds(ratings_file = 'data/ml-latest-small/ratings_noh.csv', reader=reader)

# 전체 데이터를 학습데이터로 생성함
trainset = data_folds.build_full_trainset()

# 모델 만들기
algo = SVD(n_epochs=20, n_factors=50, random_state=0)
algo.fit(trainset)

- 모델 테스트

In [39]:
ratings = pd.read_csv('data/ml-latest-small/ratings.csv') # 컬럼 존재
movies = pd.read_csv('data/ml-latest-small/movies.csv')

movieIds = ratings[ratings['userId'] == 9]['movieId']
if movieIds[movieIds==42].count() == 0:
    print('사용자 아이디 9는 영화 아이디 42의 평점 없음')

print(movies[movies['movieId']==42])

사용자 아이디 9는 영화 아이디 42의 평점 없음
    movieId                   title              genres
38       42  Dead Presidents (1995)  Action|Crime|Drama


In [40]:
# 예시 
uid = str(9)  # 숫자를 문자열로 형변환
iid = str(42)  # 숫자를 문자열로 형변환
pred = algo.predict(uid, iid)
print(pred)

user: 9          item: 42         r_ui = None   est = 3.13   {'was_impossible': False}


# 아직 보지 않은 모든 영화 예측 
- 특정 사용자가 본 영화 목록 존재
- 전체 영화 목록 존재

In [47]:
def get_unseen_movies(ratings, movies, userId):
    seen_movies = ratings[ratings['userId'] == userId]['movieId'].tolist()
    total_movies = movies['movieId'].tolist()
    
    # 모든 영화들의 movieId 중 이미 평점을 매긴 영화의 movieId를 제외하여 리스트로 생성
    unseen_movies = [movie for movie in total_movies if movie not in seen_movies]
    print('평점 매긴 영화수:',len(seen_movies), '추천대상 영화수:',len(unseen_movies), \
          '전체 영화수:',len(total_movies))
    return unseen_movies

unseen_movies = get_unseen_movies(ratings, movies, 9)

평점 매긴 영화수: 46 추천대상 영화수: 9696 전체 영화수: 9742


# 예측

In [53]:
userId = 12
unseen_movies = get_unseen_movies(ratings, movies, userId)
predictions = [algo.predict(str(userId), str(movieId)) for movieId in unseen_movies]

# 미션 : DataFrame으로 만들어서 CSV로 내보내기 결과 내보내기
# 필드명 : 사용자명, 영화제목, 예측평점
# 

평점 매긴 영화수: 32 추천대상 영화수: 9710 전체 영화수: 9742


[Prediction(uid='12', iid='1', r_ui=None, est=4.9634699968292235, details={'was_impossible': False}),
 Prediction(uid='12', iid='2', r_ui=None, est=4.441149871169427, details={'was_impossible': False}),
 Prediction(uid='12', iid='3', r_ui=None, est=4.101890497903253, details={'was_impossible': False}),
 Prediction(uid='12', iid='4', r_ui=None, est=3.95179673606818, details={'was_impossible': False}),
 Prediction(uid='12', iid='5', r_ui=None, est=3.909789011257087, details={'was_impossible': False}),
 Prediction(uid='12', iid='6', r_ui=None, est=4.891645645907925, details={'was_impossible': False}),
 Prediction(uid='12', iid='7', r_ui=None, est=3.859949824627277, details={'was_impossible': False}),
 Prediction(uid='12', iid='8', r_ui=None, est=4.111049035100181, details={'was_impossible': False}),
 Prediction(uid='12', iid='9', r_ui=None, est=3.87504555805796, details={'was_impossible': False}),
 Prediction(uid='12', iid='10', r_ui=None, est=4.404266835448777, details={'was_impossible':